In [1]:
# we load the things!

from ngsolve import *
#from netgen.csg import * 
from netgen.geom2d import SplineGeometry

from ngsolve.meshes import MakeStructured2DMesh

from netgen.occ import *
#from netgen.meshing import *

import scipy.sparse as sp
from scipy.optimize import curve_fit
import numpy as np

import pandas as pd

In [2]:
# some helper functions
def logspace_custom_decades(start, stop, points_per_decade):
    
    result = []
    current_decade = start
    while current_decade < stop:
        decade_points = np.logspace(np.log10(current_decade), np.log10(current_decade * 10), points_per_decade, endpoint=False)
        result.extend(decade_points)
        current_decade *= 10
    return np.array(result)

In [3]:
# functions for differential operators on manufactured solutions 

coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

In [4]:
# Functions for plotting, linear regression fit line for convergence

def reference_line_func(h_values, scaling_factor, slope):

    return scaling_factor * h_values ** slope

def fit_reference_line(h_values, error_values):

    popt, _ = curve_fit(reference_line_func, h_values, error_values, p0=[1, 1])

    scaling_factor, slope = popt
    return scaling_factor, slope

In [5]:
# Functions to calculate h_max

def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

def squared_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.sum((v1 - v2) ** 2)

def cayley_menger_matrix(vertices):
    if len(vertices) != 4:
        raise ValueError("This method is for a tetrahedron, which requires exactly 4 vertices.")

    # Create the Cayley-Menger matrix (5x5)
    C = np.ones((5, 5))
    for i in range(5):
        C[i, i] = 0 

    for i in range(1, 5):
        for j in range(i+1, 5):
            C[i, j] = C[j, i] = squared_distance(vertices[i-1], vertices[j-1])

    return C

def triangle_area(a, b, c):
    s = (a + b + c) / 2  
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

def circumradius_3D(vertices):
    C = cayley_menger_matrix(vertices)

    try:
        C_inv = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        raise ValueError("Cayley-Menger matrix is singular or not invertible.")

    M = -2 * C_inv
    circumradius = 0.5 * np.sqrt(M[0, 0])

    return circumradius

def calc_hmax(mesh):
    max_h = 0 
    if mesh.dim == 2:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            circumradius = circumradius_2D(a, b, c)
            max_h = max(max_h, circumradius)
    
    elif mesh.dim == 3:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            circumradius = circumradius_3D(vertices)
            max_h = max(max_h, circumradius)
    
    return max_h

def L2errorVOL(u, u_h, mesh, c=1):
    """Function to compute the L2 error in the volume, c=1 by default"""
    errorVOL = Integrate(c*(u - u_h)**2*dx, mesh)
    return errorVOL

def L2errorBND(u, u_h, mesh, c=1):
    """Function to compute the L2 error on the boundary with skeleton=True, c=1 by default"""
    dS = ds(skeleton =True, definedon=mesh.Boundaries(".*"))
    errorBND = Integrate(c*(u - u_h)**2*dS, mesh)
    return errorBND
    

In [6]:
# Create Geometry function

def createGeometry(hmax):

    wp = WorkPlane()

    wp.MoveTo(0, 0)      
    wp.LineTo(0, 0.5)
    wp.LineTo(-0.5, 0.5)  
    wp.LineTo(-0.5, -0.5)
    wp.LineTo(0.5, -0.5)
    wp.LineTo(0.5, 0)
    wp.LineTo(0, 0)
    wp.Close()

    shape = wp.Face()
    geometry = OCCGeometry(shape, dim=2)
    reEntrantCornerGeo = Mesh(geometry.GenerateMesh(maxh=hmax))
    
    return reEntrantCornerGeo

# def createGeometry(n):
#     structuredMeshUnitSquare = MakeStructured2DMesh(quads=False, nx=n, ny=n)
#     return structuredMeshUnitSquare

In [7]:
# Hodge Laplace for 1-forms function

def hodgeLaplace1Forms(mesh,
                       g = None, # this is the manufactured solution, when none is given we set it to the zero solution
                       order = 1,
                       C_w = 1):
    
    if g is None:
        g = CF((0,0))

    H_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl) space
    H_1 = H1(mesh, order=order)     # For 0-forms, H1 space
    fes = H_1 * H_curl
    (p, u), (q, v) = fes.TnT()

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size
    t = specialcf.tangential(mesh.dim)
    dS = ds(skeleton=True, definedon=mesh.Boundaries(".*"))

    f = CF(GCurl(GCurl(g)) - GGrad(GDiv(g)))                             
        
    gamma_n_u = -curl(u)*t
    gamma_n_v = -curl(v)*t

    gamma_p_v = v - n*(v*n)
    gamma_p_u = u - n*(u*n)
    gamma_p_g = g - n*(g*n)

    B, F  = BilinearForm(fes), LinearForm(fes)

    B +=  curl(u) * curl(v) * dx
    B +=  grad(p) * v * dx
    B += u * grad(q) * dx
    B += - p * q * dx

    B += gamma_n_v * gamma_p_u * dS
    B += gamma_p_v * gamma_n_u * dS
    B += (C_w/h) * gamma_p_v * gamma_p_u * dS

    F += f * v * dx
    F +=  (C_w / h) * gamma_p_g * gamma_p_v * dS
    F +=  gamma_n_v * gamma_p_g * dS
    F +=  (g*n) * q * ds
    
    with TaskManager(): 
        B.Assemble()
        F.Assemble()
        sol = GridFunction(fes)
        res = F.vec-B.mat * sol.vec
        inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    
    sol.vec.data += inv * res
    
    gf_p , gf_u = sol.components

    # Computation of all quantities needed to derive errors
    curl_u = curl(gf_u)
    grad_p = grad(gf_p)
    curl_g = CF(GCurl(g))
    p_m = - CF(GDiv(g))
    grad_p_m = CF(GGrad(p_m))
    gf_gamma_p_u = CF((gf_u - n*(gf_u*n)))
    gf_gamma_p_g = CF((g - n*(g*n)))
    gf_gamma_n_u = BoundaryFromVolumeCF(curl_u)
    gf_gamma_n_g = BoundaryFromVolumeCF(curl_g)
    gf_u_n = CF(gf_u * n)
    gf_g_n = CF(g * n)
    h_avg = 1 / Integrate(1, mesh, VOL) * Integrate(h, mesh, VOL)
    # Actual error evaluation
    # Computation of L2 errors in the volume
    E_u = L2errorVOL(gf_u, g, mesh)
    E_curl_u = L2errorVOL(curl_u, curl_g, mesh)
    E_H_curl_u = E_u + E_curl_u
    E_p = L2errorVOL(gf_p, p_m, mesh)
    E_grad_p = L2errorVOL(grad_p, grad_p_m, mesh)
    # Computation of L2 errors on the boundary
    E_gamma_p_u = L2errorBND(gf_gamma_p_u, gf_gamma_p_g, mesh)
    E_gamma_n_u = L2errorBND(gf_gamma_n_u, gf_gamma_n_g, mesh)
    E_u_n_Gamma = L2errorBND(gf_u_n, gf_g_n, mesh)
    # Hashtag and X Error norm
    HT_E_gamma_p_u = h_avg**(-1)*E_gamma_p_u
    HT_E_gamma_n_u = h_avg*E_gamma_n_u
    HT_E_u = E_H_curl_u + HT_E_gamma_p_u + HT_E_gamma_n_u
    E_h_grad_p = h_avg**2 * E_grad_p
    X_E_u_p = HT_E_u + E_p + E_h_grad_p
    print(sqrt(E_u))
    return (fes.ndof, Norm(res), 
            sqrt(E_u), sqrt(E_curl_u), sqrt(E_H_curl_u), 
            sqrt(E_p), sqrt(E_grad_p), 
            sqrt(E_gamma_p_u), sqrt(E_u_n_Gamma), 
            sqrt(HT_E_gamma_p_u), sqrt(HT_E_gamma_n_u), sqrt(HT_E_u), sqrt(E_h_grad_p),
            sqrt(X_E_u_p))

In [8]:
# definition of manufactured solutions

#omega_m_sinusoids_2D = CF((-sin(pi*x)*cos(pi*y), cos(pi*x)*sin(pi*y))) # This one vanishes!
#omega_m_sinusoids_2D = CF((sin(pi*x)*sin(pi*y), sin(pi*x)*sin(pi*y))) # This one vanishes!
#g = CF((sin(x)*cos(y)*x*(x-1)*y*(y-1), sin(y)*cos(x)*x*(x-1)*y*(y-1))) # This is a manufactured solution that DOESNT vanish on the boundary!
g = CF((sin(x)*cos(y), -sin(y)*(x)))

# A = 0.05  # Pulse Amplitude
# sigma_pulse = 0.05  # Pulse Width
# r0 = (0.5, 0.5)  # Pulse Center
# n_pulse = (1, 0)  # Pulse Direction

# omega_m_gauss_2D = CF((
#     A * exp(-((x - r0[0])**2 + (y - r0[1])**2) / (2 * sigma_pulse**2)) * n_pulse[0],
#     A * exp(-((x - r0[0])**2 + (y - r0[1])**2) / (2 * sigma_pulse**2)) * n_pulse[1]
# ))


In [9]:
# Convergence study data generation for 2D

saveBigCSV = True

refinementVals = 4
hmax_init = 0.5

maxh_values = [] 

Cw_vals = logspace_custom_decades(10**0, 100, 10)
#Cw_vals = np.linspace(2,12,20)

orders = [1, 2, 3, 4, 5]

all_results = []

for i in range(refinementVals):
    
    #print(f"hmax after refinement {refinement_step}: {h_max_eval}")
    if (i == 0):
        mesh = createGeometry(hmax_init)
    else:
        mesh.Refine()
        
    h_max_eval = calc_hmax(mesh)
    maxh_values.append(h_max_eval)
    print("doing h_max: ", h_max_eval)

    for order_cw in orders:
        results_cw = []
        print("doing order: ", order_cw)

        for C_w in Cw_vals:
            ndof, res, E_u, E_curl_u, E_H_curl_u, \
            E_p, E_grad_p, \
            E_gamma_p_u, E_u_n_Gamma, \
            HT_E_gamma_p_u, HT_E_gamma_n_u, HT_E_u, E_h_grad_p, X_E_u_p = hodgeLaplace1Forms(
                mesh, g=g, order=order_cw, C_w=C_w
            )

            row_dict = {
                'order': order_cw,
                'hmax': h_max_eval,
                'C_w': C_w,
                'ndof': ndof,
                'res': res,
                'E_u': E_u,
                'E_curl_u': E_curl_u,
                'E_H_curl_u': E_H_curl_u,
                'E_p': E_p,
                'E_grad_p': E_grad_p,
                'E_gamma_p_u': E_gamma_p_u,
                'E_u_n_Gamma': E_u_n_Gamma,
                'HT_E_gamma_p_u': HT_E_gamma_p_u,
                'HT_E_gamma_n_u': HT_E_gamma_n_u,
                'HT_E_u': HT_E_u,
                'E_h_grad_p': E_h_grad_p,
                'X_E_u_p': X_E_u_p
            }
            all_results.append(row_dict)

df_all_results = pd.DataFrame(all_results)

if (saveBigCSV == True):
    df_all_results.to_csv("all_2D_1forms_simulation_results.csv", index=False)

doing h_max:  0.35355339059327395
doing order:  1
1.4166635674545818
0.32065408803190526
0.19991019139957245
0.1554216990880325
0.127814964298511
0.12305666101794807
0.16301433057557238
0.12176427947825026
0.12196220950540007
0.12263115182753191
0.12347444329819691
0.12438054214079484
0.1252893346692619
0.12616424190959494
0.12698188659095216
0.12772771376073375
0.12839391321560228
0.12897809421135806
0.12948210251093165
0.12991084189102736
doing order:  2
0.02620896582602908
0.04039829011729681
0.05535255926040658
0.016835509747072194
0.011502702142910014
0.009870762854650893
0.00935155632437788
0.009956718840856162
0.009220335601247793
0.017227463174177813
0.008482127080440303
0.00856049742522766
0.008673336612507099
0.008786604844163264
0.00889555717436514
0.008997700309065016
0.00909144103852129
0.009175840354715711
0.009250509910396165
0.009315523896183574
doing order:  3
0.001232125635690685
0.0011574052843021398
0.0011473352761891592
0.0011972325982273432
0.0013531763532027134
0

In [10]:
# # Convergence study data generation for 2D

# saveBigCSV = True

# refinementVals = 3
# hmax_init = 0.5

# maxh_values = [] 

# Cw_vals = logspace_custom_decades(10**0, 100, 8)
# #Cw_vals = np.linspace(2,12,20)

# orders = [1, 2, 3, 4, 5]

# all_results = []

# for refinement_step in range(refinementVals):
    
#     #print(f"hmax after refinement {refinement_step}: {h_max_eval}")
#     if refinement_step == 0:
#         mesh = createGeometry(hmax_init)
#         mesh.Refine()
#     else:
#         mesh.Refine()

#     h_max_eval = calc_hmax(mesh)
#     maxh_values.append(h_max_eval)
#     print("doing h_max: ", h_max_eval)

#     for order_cw in orders:
#         results_cw = []
#         print("doing order: ", order_cw)

#         for C_w in Cw_vals:
#             g = omega_m_sinusoids_2D

#             ndof, res, L2_error_u, L2_error_curl_u, L2_error_p, L2_error_grad_p, L2_error_gamma_p_u, L2_error_u_n, L2_error_p_Gamma, HT_error_u, HT_error_gamma_n, HT_error_gamma_p = hodgeLaplace1Forms(
#                 mesh, g=g, order=order_cw, C_w=C_w
#             )

#             row_dict = {
#                 'order': order_cw,
#                 'hmax': h_max_eval,
#                 'C_w': C_w,
#                 'ndof': ndof,
#                 'res': res,
#                 'L2_error_u': L2_error_u,
#                 'L2_error_curl_u': L2_error_curl_u,
#                 'L2_error_p': L2_error_p,
#                 'L2_error_grad_p': L2_error_grad_p,
#                 'L2_error_gamma_p_u': L2_error_gamma_p_u,
#                 'L2_error_u_n': L2_error_u_n,
#                 'L2_error_p_Gamma': L2_error_p_Gamma,
#                 'HT_error_u': HT_error_u,
#                 'HT_error_gamma_n': HT_error_gamma_n,
#                 'HT_error_gamma_p': HT_error_gamma_p
#             }
#             all_results.append(row_dict)

# df_all_results = pd.DataFrame(all_results)

# if (saveBigCSV == True):
#     df_all_results.to_csv("all_2D_1forms_simulation_results.csv", index=False)
    